In [5]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [25]:
data1, data2, data3 = pd.read_json('./data1.json'), pd.read_json('./data2.json'), pd.read_json('./data3.json')

frames = [data1, data2, data3]

data = pd.concat(frames, sort=False)
trainSet, testSet = train_test_split(data, test_size=0.1)

regressor = xgb.XGBRegressor(
    n_estimators = 100,
    reg_lambda=1,
    gamma = 0,
    max_depth=3
)
regressor.fit(trainSet.iloc[:,:16], trainSet.iloc[:,16:])

[23:34:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [33]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns= data.columns[:-1])

,avocado,cheese,chicken,chickenTeriyaki,meatball,olives,steak,tomato,tuna,clear,freezingRain,rain,snowStorm,snowy,Day,Hour
0,0.135862,0.22336,0.008037,0.001624,0.158921,0.194234,0.003909,0.025545,0.111075,0.04595,0.0,0.032533,0.010218,0.007522,0.005733,0.035477


In [34]:
output_pred = regressor.predict(testSet.iloc[:,:16])


In [35]:
mean_squared_error(testSet.iloc[:,16:], output_pred)

14176.071628212629